# **TRANSFORM PERMUTATION TO TARGET MATRIX**

##### Let's tackle the problem of **transforming a given 3x3 matrix** into a **target matrix** by means of **permutations**, rotating the 4 possible internal 2x2 matrices. We'll start from scratch to see which data structures we'll use, trying to make the best design decisions in terms of efficiency and complexity.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

##### **Matrices** --> to work with matrices the ideal choice is to use numpy arrays

In [2]:
matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])
matrix

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

##### **PERMUTATIONS** --> we need a function that performs the rotations on the elements of the matrix.

##### The structure of the function is simple; we know there are **4 rotations** available corresponding to the **4 internal 2x2 submatrices**:

* **top left = 0**
* **top right = 1**
* **bottom left = 2**
* **bottom right = 3** 
  
##### Therefore, each rotation is given by a "code". We decided that the rotations will be clockwise, so we need to rotate the elements one position and the best way we came up with is:

- From each submatrix take the positions in order 
  * **top left**-->**top right**-->**bottom right**-->**bottom left**
  
  $$
    A =
    \begin{pmatrix}
    0 & 1\\
    3 & 2\\
    \end{pmatrix}
  $$

- Taking the values in this order and putting them into a linear data structure such as a **deque** lets us rotate elements and place them according to their new position.
  
$$
A =
\begin{pmatrix}
0 & 1 & 2 & 3
\end{pmatrix}
\;\longrightarrow\;
B =
\begin{pmatrix}
3 & 0 & 1 & 2
\end{pmatrix}
$$

##### Let's see how to perform the rotation efficiently; let's try with rotation of **submatrix 0** (top right), whose elements are at positions:
#####             **(00),(01),(11),(10)**

In [3]:
matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])
print('Matriz OG:')
print(matrix)

# Trying rotation 0 (top-left 2x2 submatrix)
positions = [(0,0),(0,1),(1,1),(1,0)]

# Put the values from those positions into 
# a Double Ended Queue (Deque)
values = deque([matrix[i] for i in positions])
print("\n", 'Valores Submatriz 0:')
print(values)

# Rotate the values one position
values.rotate(1)
print("\n", 'Valores Submatriz 0 rotados:')
print(values)

# Assign the rotated values to the positions
for i in range(len(positions)): matrix[positions[i]] = values[i]
print("\n", 'Permutación:')
print(matrix)


Matriz OG:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

 Valores Submatriz 0:
deque([np.int64(1), np.int64(2), np.int64(5), np.int64(4)])

 Valores Submatriz 0 rotados:
deque([np.int64(4), np.int64(1), np.int64(2), np.int64(5)])

 Permutación:
[[4 1 3]
 [5 2 6]
 [7 8 9]]


##### Now let's use this logic to create the **function** that with a **matrix and a given rotation code** applies that **rotation** and returns it:

In [4]:
def giro(num, matrix):
    m = matrix.copy()

    # List of positions involved in each 
    # rotation; according to the index (num) the rotation applies
    # to one submatrix or another
    position_matrix = [[(0,0),(0,1),(1,1),(1,0)],
                       [(0,1),(0,2),(1,2),(1,1)],
                       [(1,0),(1,1),(2,1),(2,0)],
                       [(1,1),(1,2),(2,2),(2,1)]]
    
    # Select the positions and store them in a deque
    positions = position_matrix[num] 
    values = deque([m[i] for i in positions])
    # print(values)
    values.rotate(1)
    # print(values)
    for i in range(len(positions)): m[positions[i]] = values[i]
    
    return m
    
    

In [5]:
matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])
print('Matriz OG:')
print(matrix)

rotada = giro(0,matrix) # Using the function
print('\nMatriz Rot0:')
print(rotada)

Matriz OG:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Matriz Rot0:
[[4 1 3]
 [5 2 6]
 [7 8 9]]


##### We should consider that, although for working with matrices the best option is **numpy arrays**, the core of this algorithm is **Breadth-First Search (BFS)**, which we intend to **optimize**.

##### To **optimize** the search means **avoiding cycles**, **avoiding exploring branches already explored**... For that we need to keep **control of the states** that have been **explored**, and therefore we must **check** in each case whether a **state has already been developed**.

##### Carrying out that control purely with arrays is very costly; it is not efficient, so we must choose another structure.

##### We will work with **tuples** (which are immutable) contained **in a set**, which internally uses their **hash** to locate them, meaning we can **check quickly** and efficiently whether a state **has already been explored or enqueued**.

##### Our logic for the function is the same, since for the rotations it is efficient; what happens is that outside of it, the states must be in tuple form.

##### Therefore, we need to **wrap it**: outside this function we will deal with tuples. A tuple will arrive at this function and another tuple must leave it, so we need to create a function that:

1. **Receives a tuple**
   
2. **Converts it to a matrix**
   
3. **Passes it to our function to execute the rotation**
   
4. **Takes the new matrix**
   
5. **Converts it to a tuple**
   
6. **Returns the new tuple**

##### First, let us see how to convert **tuples to matrices and vice versa**

In [6]:
# Original matrix (numpy array)
matrix_OG = np.array([[1,2,3],[4,5,6],[7,8,9]])

print(matrix_OG)
print()

# Array to tuple transformation
tupla = tuple(matrix_OG.flatten())
print(tupla)
print(f'Hash de la tupla: {hash(tupla)}\n')

# !! WHEN WE WANT TO CHECK IF A VALUE HAS BEEN EXPLORED OR ENQUEUED
#    WE WILL USE `state in set`, WHICH INTERNALLY USES THE TUPLE HASH
#    TO LOCATE IT !!! --> Efficient

# Tuple to array transformation
matriz2 = np.array(tupla).reshape(3,3)
print(matriz2)


[[1 2 3]
 [4 5 6]
 [7 8 9]]

(np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9))
Hash de la tupla: 2885636760529661641

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [7]:
# FUNCTIONS TO CONVERT TUPLE <--> MATRIX
# Simply a wrapper around the conversions

    # Tuple to Matrix
def ttom(tupla):
    return np.array(tupla).reshape(3,3)

    # Matrix to Tuple
def mtot(matrix):
    return tuple(matrix.flatten())

##### Now we can make the **wrapper** that uses `giro()` so it **receives tuples and returns tuples**, while working with arrays internally.

In [8]:
def giro_w(num, state):
    matrix = ttom(state)

    rotated = giro(num, matrix)

    new_state = mtot(rotated)

    return new_state

# SAME FUNCTION WITH PRINTS() TO VISUALIZE CONVERSION
def giro_w_verbose(num, state):
    print(state, '\n')

    matrix = ttom(state)
    print(matrix)

    rotated = giro(num, matrix)
    print()
    print(rotated)

    new_state = mtot(rotated)
    print()
    print(new_state)

    return new_state

In [9]:
# We create a matrix and convert it to a tuple:
matrix_OG = np.array([[1,2,3],[4,5,6],[7,8,9]])
tupla = mtot(matrix_OG)

state = giro_w_verbose(0, tupla) # Receives and returns tuples

(np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)) 

[[1 2 3]
 [4 5 6]
 [7 8 9]]

[[4 1 3]
 [5 2 6]
 [7 8 9]]

(np.int64(4), np.int64(1), np.int64(3), np.int64(5), np.int64(2), np.int64(6), np.int64(7), np.int64(8), np.int64(9))


##### We now have the tool that will generate the permutations at each moment; with it we can create the **successor generation function**, which simply must **take the current state** and return its possible **successors** by applying **each of the 4 rotations**.

#### In addition, since we intend to **recover the sequence of operations** (rotations) applied to the original matrix, we will make it **return the rotation code** (0, 1, 2, 3). 

#### This way we can **extract the sequence of rotations to reach the target matrix**

In [10]:
def gen_suc_matrix(state):

    # We return a list of tuples that contain
    # (Successor_i, rotation_i)
    return [(giro_w(i, state), i) for i in range(4)]

In [11]:
# We create a matrix and convert it to a tuple:
matrix_OG = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(matrix_OG,'\n')

tupla = mtot(matrix_OG)
print(tupla)

# We generate the successors
sucesores = gen_suc_matrix(tupla)

# We iterate taking the first element of each tuple (state)
for i in sucesores: print('\n', ttom(i[0]))

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

(np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9))

 [[4 1 3]
 [5 2 6]
 [7 8 9]]

 [[1 5 2]
 [4 6 3]
 [7 8 9]]

 [[1 2 3]
 [7 4 6]
 [8 5 9]]

 [[1 2 3]
 [4 8 5]
 [7 9 6]]


## **BFS**

##### We now have the **successor generation** tool (`gen_suc_matrix()`) and we know the **data structures** we are going to use; now we must go to the **core of the algorithm**: the **Breadth-First Search** to get from the **initial matrix** to the **target matrix**.

##### Let's gather all the code here again so we don't lose sight of it and implement the search.

In [12]:
def giro(num, matrix):
    m = np.array(matrix).copy()
    subm_positions = [[(0,0),(0,1),(1,1),(1,0)],
                       [(0,1),(0,2),(1,2),(1,1)],
                       [(1,0),(1,1),(2,1),(2,0)],
                       [(1,1),(1,2),(2,2),(2,1)]]
    
    positions = subm_positions[num]
    
    values = deque([m[i] for i in positions])
    
    values.rotate(1)
    
    for i in range(len(positions)): m[positions[i]] = values[i]
    
    return m

def giro_w(num, state):
    matrix = np.array(state).reshape(3,3)

    rotated = giro(num, matrix)

    new_matrix = tuple(rotated.flatten())

    return new_matrix

def gen_suc_matrix(state):
    return [(giro_w(i, state), i) for i in range(4)]

##### We need:

##### Define an **initial state** and a **goal state**

#####     · The **queue** that holds the **states to explore**

#####     · The **set** that holds the **explored states**

#####     · The **set** that holds the **states that have ever been enqueued**

#####     · The **dictionary** that holds the pairs **state**:**(parent, rotation)** to do **backtracking** and recover the **sequence** of **states** and **rotations** applied.

*In reality, a single set of **visited** would be enough to avoid enqueueing successors that have already been explored, but for the didactic nature of the work and in case we decide to print the queue and the set of explored states it is preferable to do it this way; otherwise, if we only had one set of visited states to avoid enqueueing, when printing the queue and visited set the progression is not as clear.*


In [13]:
# INITIAL STATE
state1 = np.array([[1,2,3],
                   [4,5,6],
                   [7,8,9]])

state1=mtot(state1)

# GOAL STATE
goal = np.array([[5,4,3],   # Just apply rotation 0 twice 
                 [2,1,6],   # times rotation 0 (top-left submatrix 
                 [7,8,9]])  # top left)
goal=mtot(goal)

# QUEUE OF STATES TO EXPLORE
cola = deque()

# SET OF ENQUEUED VALUES
on_cola = set()

# SET OF EXPLORED VALUES
explored = set()

# DICTIONARY state : (parent, rotation)
gen_tree = {}

##### We already have the **data structures** ready to use; now we need to implement the **search loop**.

In [14]:
# Enqueue the initial state
cola.append(state1)

# I think it is redundant but for some reason without this
# the loop becomes infinite
exito = False

# While there are elements in the queue and the goal is not found 
# the goal state:
while cola and not exito:
    # Pop the element to explore from the queue
    elem = cola.popleft()
    
    # Add it to the explored set (if it gets here
    # it is going to be explored)
    explored.add(elem)

    # Generate successors
    sucessors = gen_suc_matrix(elem)

    # For each successor:
    for i in sucessors:

        # If it is the goal state:
        if i[0] == goal:
            print(f'exito!: {i[0]}')
            # This ends the loop
            exito=True
            # Add the key to the dictionary
            # i[0], which is the state, and as value
            # the tuple (parent, rotation)
            gen_tree[i[0]] = (elem, i[1])

        elif i[0] not in on_cola:
            # Add the successor to the queue 
            # if it is NEITHER IN NOR HAS BEEN
            cola.append(i[0])
            on_cola.add(i[0])
            # Add the key to the dictionary
            # i[0], which is the state, and as value
            # the tuple (parent, rotation)
            gen_tree[i[0]] = (elem, i[1])


exito!: (np.int64(5), np.int64(4), np.int64(3), np.int64(2), np.int64(1), np.int64(6), np.int64(7), np.int64(8), np.int64(9))


##### We already have the search algorithm; it can reach the goal state, but we would be interested in being able to **recover the sequence of rotations** that were performed.

##### We will create a **function that wraps the previous logic** and **returns** the **succession dictionary**:

In [15]:
def matrix_BFS(init_state, goal):
   # QUEUE OF STATES TO EXPLORE
    cola = deque()

    # SET OF ENQUEUED VALUES
    on_cola = set()

    # SET OF EXPLORED VALUES
    explored = set()

    # DICTIONARY state : (parent, rotation)
    gen_tree = {}

    cola.append(state1)

    exito = False
    while cola and not exito:
        elem = cola.popleft()
        
        explored.add(elem)

        sucessors = gen_suc_matrix(elem)

        # For each successor:
        for i in sucessors:

            # If it is the goal state:
            if i[0] == goal:
                print(f'exito!: {i[0]}')
                exito=True
                gen_tree[i[0]] = (elem, i[1])

            elif i[0] not in on_cola:
                cola.append(i[0])
                on_cola.add(i[0])
                gen_tree[i[0]] = (elem, i[1])

    return gen_tree

matrix_gen_tree = matrix_BFS(state1, goal)

exito!: (np.int64(5), np.int64(4), np.int64(3), np.int64(2), np.int64(1), np.int64(6), np.int64(7), np.int64(8), np.int64(9))


In [16]:
for i,j in matrix_gen_tree.items(): print(f'\nKey: {i}\nValue:{j}')


Key: (np.int64(4), np.int64(1), np.int64(3), np.int64(5), np.int64(2), np.int64(6), np.int64(7), np.int64(8), np.int64(9))
Value:((np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)), 0)

Key: (np.int64(1), np.int64(5), np.int64(2), np.int64(4), np.int64(6), np.int64(3), np.int64(7), np.int64(8), np.int64(9))
Value:((np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)), 1)

Key: (np.int64(1), np.int64(2), np.int64(3), np.int64(7), np.int64(4), np.int64(6), np.int64(8), np.int64(5), np.int64(9))
Value:((np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)), 2)

Key: (np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(8), np.int64(5), np.int64(7), np.int64(9), np.int64(6))
Value:((np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.in

##### As we mentioned, `gen_tree` is a dictionary that holds as a **key** a **state** and as a **value** a **tuple** containing **(parent, rotation)**, that is, its **parent state and the rotation that was applied to the parent to generate it**.

##### Starting from the **goal**, which we know, we must recover the sequence backwards (**backtracking**).

In [17]:
# SEQUENCE OF STATES
states_sec = deque()

# SEQUENCE OF OPERATIONS
opps =deque()

# The first element we look for is the goal
# from there we look for its parent and so
# on successively
elem=goal

while True:
    # We insert the states on the left
    # To take them out already ordered
    states_sec.appendleft(elem)

    # If the state differs from the initial state
    # add the operation that created it and
    # move on to the next element
    if elem != state1:
        opps.appendleft(gen_tree[elem][1])
        elem = gen_tree[elem][0]

    else: break

print(f'Secuencia de Estados: {states_sec}')
print(f'Secuencia de giros: {opps}')

Secuencia de Estados: deque([(np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)), (np.int64(4), np.int64(1), np.int64(3), np.int64(5), np.int64(2), np.int64(6), np.int64(7), np.int64(8), np.int64(9)), (np.int64(5), np.int64(4), np.int64(3), np.int64(2), np.int64(1), np.int64(6), np.int64(7), np.int64(8), np.int64(9))])
Secuencia de giros: deque([0, 0])


##### Let's check

* We know that the initial state was:
$$
    State1 =
    \begin{pmatrix}
    1 & 2 & 3\\
    4 & 5 & 6\\
    7 & 8 & 9\\
    \end{pmatrix}
  $$

* The target (result of two **0 rotations**) was:
$$
    State1 =
    \begin{pmatrix}
    5 & 4 & 3\\
    2 & 1 & 6\\
    7 & 8 & 9\\
    \end{pmatrix}
  $$

##### We must check **whether applying the sequence of rotations** that the algorithm gave us **produces** the matrix:

In [18]:
resultado = giro_w(0, giro_w(0, state1))
print(resultado)
print(resultado == goal)

(np.int64(5), np.int64(4), np.int64(3), np.int64(2), np.int64(1), np.int64(6), np.int64(7), np.int64(8), np.int64(9))
True


##### Let's create a function for **backtracking** that **returns the sequence of rotations** and include it in `matrix_BFS()` so that instead of returning the succession dictionary it returns the sequence.

In [19]:
def backtrack(gen_tree):
    # SEQUENCE OF STATES
    states_sec = deque()

    # SEQUENCE OF OPERATIONS
    opps =deque()

    elem=goal

    while True:
    
        states_sec.appendleft(elem)

        if elem != state1:
            opps.appendleft(gen_tree[elem][1])
            elem = gen_tree[elem][0]

        else: break
    
    return opps

secuencia_ops = backtrack(gen_tree)
print(f'Secuencia de Estados: {secuencia_ops}')


Secuencia de Estados: deque([0, 0])


In [20]:
# MODIFICATION matrix_BFS()
def matrix_BFS(init_state, goal):
   # QUEUE OF STATES TO EXPLORE
    cola = deque()

    # SET OF ENQUEUED VALUES
    on_cola = set()

    # SET OF EXPLORED VALUES
    explored = set()

    # DICTIONARY state : (parent, rotation)
    gen_tree = {}

    cola.append(state1)

    exito = False
    while cola and not exito:
        elem = cola.popleft()
        
        explored.add(elem)

        sucessors = gen_suc_matrix(elem)

        # For each successor:
        for i in sucessors:

            # If it is the goal state:
            if i[0] == goal:
                # print(f'exito!: {i[0]}')
                exito=True
                gen_tree[i[0]] = (elem, i[1])

            elif i[0] not in on_cola:
                cola.append(i[0])
                on_cola.add(i[0])
                gen_tree[i[0]] = (elem, i[1])

    op_sec = backtrack(gen_tree)
    return op_sec

ops = matrix_BFS(state1, goal)
print(ops)

deque([0, 0])


##### Function to check that starting from an **initial state** and a **given sequence of rotations** we reach the **goal**

In [21]:
def rep_ops(state, op_sec): # Function that applies the sequence of operations to the initial element
                            # So we can verify whether the sequence is correct
    for i in op_sec:
        state = giro_w(i, state)
            
    return state

def check_ops_sec(state, op_sec, goal): # Checks whether the initial state after applying the sequence
                                        # of rotations equals the goal.
    return goal == rep_ops(state, op_sec)

check_ops_sec(state1, opps, goal)

True

### **FULL APPLICATION**

In [22]:
# Generate random permutation of state1
import random as rd
print(f'Matriz OG:\n{ttom(state1)}\n')

goal = state1
for i in range(100): goal = giro_w(rd.randint(0,3), goal)

print(f'Random Goal:\n{ttom(goal)}\n')

Matriz OG:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Random Goal:
[[2 9 3]
 [1 6 7]
 [4 8 5]]



In [23]:
ops_sec = matrix_BFS(state1, goal)
print(f'Secuencia Giros: {ops_sec}')

Secuencia Giros: deque([3, 0, 2, 2, 3, 0, 2, 0, 2, 2])


In [24]:
print(f'Secuencia de Giros correcta?: {check_ops_sec(state1, ops_sec, goal)}')

Secuencia de Giros correcta?: True


# END